In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate , train_test_split , StratifiedShuffleSplit
from sklearn import   linear_model, metrics
from sklearn.linear_model import SGDClassifier as SGD

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.figure_factory as ff

In [2]:
train_directory = "data/train.csv"

train = pd.read_csv(train_directory, header=0 )

train['y'] = [1 if y == 'yes' else 0 for y in train['y']]

In [3]:
X = train[['age' , 'previous']][:]
y = train['y'][:]

In [4]:
classifier = RandomForestClassifier()

In [5]:
train_data, test_data, train_labels, test_labels = train_test_split(X, y, 
                                                                                     test_size = 0.2)

In [6]:
classifier.fit(train_data , train_labels)

C:\Users\User\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [7]:
pred_labels = classifier.predict(test_data)

In [8]:
metrics.roc_auc_score(test_labels, pred_labels)

0.5151995883891437

# 1. Выводы по существующим данным и исходной модели

В случае исходной модели обрабатывается только один признак - количество предыдущих платежей. Для большого количества данных одного признака недостаточно, поэтому и была получена плохая оценка качества модели ~0,5, что говорит о том, что модель предсказывает по методу "пальцем в небо". Сначала необходимо добавить больше признаков для нашей модели. 

Всего в сете 20 признаков + таргет, поэтому есть где разгуляться и добавить новые признаки

In [9]:
train.shape
print('Имеющиеся признаки: ', '; '.join(list(train)))
print('Наличие null ', train.isnull().values.any())
print('Наличие пустых значений ', train.isna().values.any())
train.head()

Имеющиеся признаки:  age; job; marital; education; default; housing; loan; contact; month; day_of_week; duration; campaign; pdays; previous; poutcome; emp.var.rate; cons.price.idx; cons.conf.idx; euribor3m; nr.employed; y
Наличие null  False
Наличие пустых значений  False


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,blue-collar,married,basic.9y,no,yes,no,cellular,may,fri,...,2,999,0,nonexistent,-1.8,92.893,-46.2,1.313,5099.1,0
1,39,services,single,high.school,no,no,no,telephone,may,fri,...,4,999,0,nonexistent,1.1,93.994,-36.4,4.855,5191.0,0
2,25,services,married,high.school,no,yes,no,telephone,jun,wed,...,1,999,0,nonexistent,1.4,94.465,-41.8,4.962,5228.1,0
3,38,services,married,basic.9y,no,unknown,unknown,telephone,jun,fri,...,3,999,0,nonexistent,1.4,94.465,-41.8,4.959,5228.1,0
4,47,admin.,married,university.degree,no,yes,no,cellular,nov,mon,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.191,5195.8,0


# 2. Подготовка данных
Многие признаки имеют нечисловые значения, которые не могут обрабатываться. Поэтому нужно перевести их в числа

In [10]:
for feature in list(train):
    values = set(train[feature])
    print(feature.upper(), ':', values)
    print()


AGE : {18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 85, 86, 88}

JOB : {'student', 'blue-collar', 'management', 'self-employed', 'entrepreneur', 'retired', 'admin.', 'unknown', 'unemployed', 'housemaid', 'technician', 'services'}

MARITAL : {'married', 'single', 'unknown', 'divorced'}

EDUCATION : {'basic.9y', 'basic.4y', 'high.school', 'basic.6y', 'unknown', 'professional.course', 'university.degree'}

DEFAULT : {'unknown', 'no'}

HOUSING : {'yes', 'unknown', 'no'}

LOAN : {'unknown', 'no', 'yes'}

CONTACT : {'telephone', 'cellular'}

MONTH : {'may', 'oct', 'jul', 'jun', 'mar', 'dec', 'sep', 'nov', 'apr', 'aug'}

DAY_OF_WEEK : {'fri', 'thu', 'mon', 'wed', 'tue'}

DURATION : {0, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,

Конечное множество строковых значений принимают фичи: job, marital, education, default, housing, loan, contact, month, day_of_week, poutcome. 

Пронумеруем все значения в множестве и заменим на порядковый номер все значения. За 0 примем во всех фичах значение 'unknown'

In [11]:
# замена 'job'
job_values = ['technician', 'services', 'self-employed',
              'admin.', 'retired', 'management', 
              'blue-collar', 'housemaid', 'unemployed',
              'student', 'entrepreneur', 'unknown']

job_numbers = [1, 2, 3,
              4, 5, 6,
              7, 8, 9,
              10, 11, 0]

train['job'] = train['job'].replace(job_values, job_numbers)

# замена 'marital'
marital_values = ['unknown', 'married', 'single', 'divorced']
marital_numbers = [0, 1, 2, 3]

train['marital'] = train['marital'].replace(marital_values, marital_numbers)

# замена 'education'
edu_values = ['basic.4y', 'basic.6y', 'basic.9y',
              'high.school',  'professional.course', 'university.degree', 'unknown']
edu_numbers = [1, 2, 3, 4, 5, 6, 0]

train['education'] = train['education'].replace(edu_values, edu_numbers)

# замена 'default'
default_values = ['no', 'unknown']
default_numbers = [1, 0]

train['default'] = train['default'].replace(default_values, default_numbers)

# замена 'housing'
house_values = ['no', 'yes', 'unknown']
house_numbers = [1, 2, 0]

train['housing'] = train['housing'].replace(house_values, house_numbers)

# замена 'loan'
loan_values = ['no', 'yes', 'unknown']
loan_numbers = [1, 2, 0]

train['loan'] = train['loan'].replace(loan_values, loan_numbers)

# замена 'contact'
contact_values = ['cellular', 'telephone']
contact_numbers = [1, 2]

train['contact'] = train['contact'].replace(contact_values, contact_numbers)

# замена 'month'
# возьмем все месяцы, даже если их нет в выборке
month_values = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
month_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

train['month'] = train['month'].replace(month_values, month_numbers)

# замена 'day_of_week'
# возьмем все дни, даже если их нет в выборке
day_values = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']
day_numbers = [1, 2, 3, 4, 5, 6, 7]

train['day_of_week'] = train['day_of_week'].replace(day_values, day_numbers)

# замена 'poutcome'
outcome_values = ['success', 'nonexistent', 'failure']
outcome_numbers = [1, 0, -1]

train['poutcome'] = train['poutcome'].replace(outcome_values, outcome_numbers)

train.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,30,7,1,3,1,2,1,1,5,5,...,2,999,0,0,-1.8,92.893,-46.2,1.313,5099.1,0
1,39,2,2,4,1,1,1,2,5,5,...,4,999,0,0,1.1,93.994,-36.4,4.855,5191.0,0
2,25,2,1,4,1,2,1,2,6,3,...,1,999,0,0,1.4,94.465,-41.8,4.962,5228.1,0
3,38,2,1,3,1,0,0,2,6,5,...,3,999,0,0,1.4,94.465,-41.8,4.959,5228.1,0
4,47,4,1,6,1,2,1,1,11,1,...,1,999,0,0,-0.1,93.200,-42.0,4.191,5195.8,0


# 3. Попытки улучшить модель
## 3.1. Методом "в лоб"
Возьмем все имеющиеся признаки (почему бы и нет, если их количество не сильно велико), а затем будем постепенно их удалять и наблюдать за изменением качества модели 

In [27]:
X = train[:][:]
X = X.drop('y', axis=1)
y = train['y'][:]

In [13]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

scores_rf = cross_validate(classifier, X, y, scoring='roc_auc', cv=sss)

scores_rf['test_score'].mean()

0.8968424541819038

In [14]:
sgd = SGD(max_iter=1000, tol=1e-3)

scores_sgd = cross_validate(sgd, X, y, scoring='roc_auc', cv=sss)
scores_sgd['test_score'].mean()

0.8825816385449414

## 3.2. Фильтрация признаков

### 3.2.1. Проверка зависимости таргета от признака
Определим влияние признака на значение целевого параметра

In [15]:
features = list(train)
features.remove('y')
target = list(train['y'])

corr_values = {}
for feature in features:      
    corr_values[feature] = train['y'].astype('float64').corr(train[feature].astype('float64'))

data = [go.Bar(y=list(corr_values.values()), x=list(corr_values.keys()))]

iplot(data)

Большая (по модулю) корреляция обнаружена на признаках duration, pdays, previous, emp.var.rate, euribor3m и nr.employed. Попробуем составить классификацию по этим признакам

In [32]:
X = train[['duration', 'pdays', 'previous', 'emp.var.rate', 'euribor3m', 'nr.employed']]
y = train['y']

In [17]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

scores_rf = cross_validate(classifier, X, y, scoring='roc_auc', cv=sss)

scores_rf['test_score'].mean()

0.8705972926156413

In [18]:
sgd = SGD(max_iter=1000, tol=1e-3)

scores_sgd = cross_validate(sgd, X, y, scoring='roc_auc', cv=sss)
scores_sgd['test_score'].mean()

0.8814546309959154

### 3.2.2. Определение взаимосвязанных признаков

In [35]:
X = train[:][:]
X = X.drop('y', axis=1)
y = train['y'][:]

corr_features = X.corr()
corr_features.head()

,age,job,marital,education,default,housing,loan,contact,month,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
age,1.000000,0.032453,-0.120606,-0.167913,-0.166931,-0.007873,0.001910,0.024893,0.036197,0.039014,-0.006501,-0.040607,0.044000,0.023187,-0.011676,0.008798,0.086147,-0.009824,-0.036183
job,0.032453,1.000000,-0.047286,-0.273679,-0.094983,0.011255,0.000275,0.050780,0.008703,0.021832,-0.053307,0.007647,0.005273,-0.018442,-0.027801,0.030308,-0.064201,-0.025498,-0.026933
marital,-0.120606,-0.047286,1.000000,0.074236,0.091265,0.029736,0.006990,-0.060635,0.014228,0.015447,0.003189,-0.016277,0.037648,-0.024024,-0.053043,-0.027586,-0.044862,-0.057024,-0.052470
education,-0.167913,-0.273679,0.074236,1.000000,0.235561,0.054947,0.011536,-0.132811,0.134556,-0.005817,0.024268,-0.043201,0.032723,0.023182,-0.050796,-0.103538,0.070627,-0.038001,-0.038499
default,-0.166931,-0.094983,0.091265,0.235561,1.000000,0.016804,-0.011074,-0.157966,0.114282,0.017884,-0.017215,-0.091164,0.095020,0.000103,-0.187312,-0.164665,-0.024002,-0.180920,-0.172365


In [37]:
figure = ff.create_annotated_heatmap(
    z=corr_features.values,
    x=list(corr_features.columns),
    y=list(corr_features.index),
    annotation_text=corr_features.round(2).values,
    showscale=True)

iplot(figure)

Заметим корреляцию больше 0,7 по модулю среди средующих параметров:
1. emp.var.rate - nr.employed
2. emp.var.rate - euribor3m
3. emp.var.rate - cons.price.idx
4. nr.employed - euribor3m

То есть, эти признаки взаимосвязаны, поэтому они вместе будут оказывать сильное воздействие на модель. Будем среди пар признаков оставлять только один. В результате останеся только признак emp.var.rate

In [39]:
X = train[:][:]
X = X.drop(['y', 'nr.employed', 'euribor3m', 'cons.price.idx'], axis=1)
y = train['y'][:]

In [40]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

scores_rf = cross_validate(classifier, X, y, scoring='roc_auc', cv=sss)

scores_rf['test_score'].mean()

0.8958736981672761

In [41]:
sgd = SGD(max_iter=1000, tol=1e-3)

scores_sgd = cross_validate(sgd, X, y, scoring='roc_auc', cv=sss)
scores_sgd['test_score'].mean()

0.888415559975193

### 3.2.3. Объединение методов 3.2.1 и 3.2.2

В первом случае признаки duration, pdays, previous, emp.var.rate, euribor3m и nr.employed были определены как наиболее связанные с таргетом. Второй случай определелил, что среди этих признаков есть взаимосвязанные:
1. emp.var.rate - nr.employed
2. emp.var.rate - euribor3m
3. nr.employed - euribor3m

Поэтому среди этих признаков можно оставить duration, pdays, previous и emp.var.rate

In [43]:
X = train[['duration', 'pdays', 'previous','emp.var.rate']]
y = train['y']

In [44]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

scores_rf = cross_validate(classifier, X, y, scoring='roc_auc', cv=sss)

scores_rf['test_score'].mean()

0.8236853360706572

In [45]:
sgd = SGD(max_iter=1000, tol=1e-3)

scores_sgd = cross_validate(sgd, X, y, scoring='roc_auc', cv=sss)
scores_sgd['test_score'].mean()

0.8879728833857274

### 3.2.4. Простая логика

Предыдущие модели были построены путем вычислений. Вроде бы получаются неплохие значения (если я все правильно делаю). Теперь попробуем отобрать признаки, используя свою логику

Итак, у нас есть следующие признаки: 
1. age - возраст пользователя
2. job - категория вида деятельности
3. marital - семейный статус
4. education - образование (высшее из имеющихся)
5. housing - наличие недвижимости в собственности
6. loan - наличие других кредитов
7. contact - тип контакта (городской или мобильный)
8. month - месяц, в котором подана заявка на кредит?
9. day_of_week - день недели, в который подана заявка на кредит?
10. duration - длительность займа?
11. campaign - компания, в которой работет пользователь?
12. previous - количество предыдущих заемов
13. poutcome - исход предыдущих заемов

Смысл остальных признаков я определить не смогла.

В данном случае я считаю, что наиболее существенными признаками будут:
1. age - как правило, молодые люди не могут иметь постоянного большого дохода, чтобы суметь стабильно оплачивать заем, в свою очередь люди в возрасте имеют риск потерять работу (здоровье, пенсия и т.п), поэтому это важный фактор
2. job - разные типы работы оплачиваются по-разному. Соответственно, платежеспособность будет разной, поэтому стабильность выбоат и возможность уплаты заема есть у людей со стабильным высоким доходом
3. marital - здесь спорный вопрос. С одной стороны, семейные люди могут поддерживать кредит супруга даже в критических ситуациях (здоровье, потеря работы, смерть и т.д.). с другой - семейные люди основную часть доходов будут тратить на семью, детей и т.д., а люди без пары могут ратить существенную часть доходов на оплату. Так или иначе, признак будет влять
4. education - однозначно, чем выше у человека образование, тем более стабильный и высокий доход он будет иметь (не будем рассматривать Джобсов, Гейтсов и т.п.)
5. housing - при наличии недвижимости в случае неуплат долга можно изъять у человека недвижимость. Если у пользователя она есть, то нам есть что изымать, в противном случае у ччеловека нечего забрать в случае неуплаты
6. loan - если у человека уже есть долги, то он может несправиться с большим количеством уплат, поэтому наличие долгов сильно влияет на платежеспособность
7. duration (если это длительность займа) - чем долше длительность займа, тем труднее предсказат поведения пользователя в течение периода. Если длительность заема большая, то за этот период может случиться много факторов, меняющих платежеспособность пользователя
8. previous - если у человека есть предыдущие платежи, то можно ознакомиться с его кредитной историей, если о человеке ничего не известно, то трудно спрогнозироваь его поведение
9. poutcome - если пользователь успешно закрыл предыдущие долги, то ему можно доверять

In [47]:
X = train[['age', 'job', 'marital', 'education', 'housing', 'loan', 'duration', 'previous','poutcome']]
y = train['y']

In [48]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

scores_rf = cross_validate(classifier, X, y, scoring='roc_auc', cv=sss)

scores_rf['test_score'].mean()

0.8158636470563077

In [49]:
sgd = SGD(max_iter=1000, tol=1e-3)

scores_sgd = cross_validate(sgd, X, y, scoring='roc_auc', cv=sss)
scores_sgd['test_score'].mean()

0.4310215778105686

Логика не помогла ((((

Точность модели уменьшилась, поскольку мы не использовали факторы, оказывающие влияние на таргет (факторы, полученные в методе 3.2.1)

Зато можно сделать вывод, что в прогнозировании нельзя смотреть поверхностно. Необходимо искать скрытые и/или не всегда очевидные факторы, которые впоследствии смогут оказать воздействие на результат

# 4. Итог

В результате были получены показатели roc-auc на тесте 0,88 на RandomForest и SGD 

Если бы было больше времени, то можно было бы попробовать определить нелинейные зависимости между таргетом и признаком, поскольку используемый метод определения значимости показывал только линейную зависимость. Можно было бы поиграть с количеством деревьев в RandomForest, поиграть с количеством факторов: прямо построить графики, как меняется ошибка в зависимости от количества деревьев и факторов. Попробовать другие методы определения значимых факторов (гугл в помощь)